In [1]:
import requests
from urllib.parse import urlencode
from urllib.parse import urlparse, parse_qsl
from _kluce import api_key as GOOGLE_API_KEY

Google Maps Client API

In [2]:
class GoogleMapsClient(object):
    lat = None
    lng = None
    data_type = 'json'
    location_query = None
    api_key = None
    
    def __init__ (self, api_key=None, asdress_or_postal_code=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if api_key == None:
            raise Exception('API key is required')
        self.api_key = api_key
        self.location_query = asdress_or_postal_code
        if self.location_query != None:
            self.extract_lat_lng()
    
    def extract_lat_lng(self, location=None):
        loc_query = self.location_query
        if location != None:
            loc_query=location
        endpoint = f"https://maps.googleapis.com/maps/api/geocode/{self.data_type}"
        params = {"address": loc_query, "key": self.api_key}
        url_params = urlencode(params)

        url = f"{endpoint}?{url_params}"
        r = requests.get(url)
        if r.status_code not in range (200,299):
            return {}
        latlng = {}
        try:
            latlng = r.json()['results'][0]['geometry']['location']
        except:
            pass
        lat, lng = latlng.get("lat"), latlng.get("lng")
        self.lat = lat
        self.lng = lng
        return lat, lng
    
    def search(self, keyword="China Food", radius=5000,  location = None):
        lat, lng = self.lat, self.lng
        if location != None:
            lat, lng = self.extract_lat_lng(location)
        endpoint = f'https://maps.googleapis.com/maps/api/place/nearbysearch/{self.data_type}'
        params = {
            "key": self.api_key,
            "location": f"{lat},{lng}",
            "radius": radius,
            "keyword": keyword
        }
        params_encoded = urlencode(params)
        places_url = f"{endpoint}?{params_encoded}"
        r = requests.get(places_url)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def detail(self, place_id='ChIJB8F-_QHFw0cRkIcY_Zn5M04', fields=["name", "rating", "formatted_phone_number", "formatted_address"]):
        #  Detail lookup
        detail_base_endpoint = f'https://maps.googleapis.com/maps/api/place/details/{self.data_type}'
        detail_params = {
            "place_id": f"{place_id}",
            "fields": ",".join(fields),
            "key": self.api_key
        }
        detail_params_encoded = urlencode(detail_params)
        detail_url = f"{detail_base_endpoint}?{detail_params_encoded}"
        r = requests.get(detail_url)
        if r.status_code not in range(200,299):
            return {}
        return r.json()

In [3]:
client = GoogleMapsClient(api_key=GOOGLE_API_KEY, asdress_or_postal_code='163, avenue de Fre, uccle')
print(client.lat, client.lng)

50.8041613 4.3563086


In [10]:
client.search("italien restaurant", radius = 500)

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 50.80598120000001, 'lng': 4.3577732},
    'viewport': {'northeast': {'lat': 50.80733182989272,
      'lng': 4.359135729892722},
     'southwest': {'lat': 50.80463217010728, 'lng': 4.356436070107277}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
   'icon_background_color': '#FF9E67',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
   'name': 'I Monticelli',
   'opening_hours': {'open_now': False},
   'photos': [{'height': 1960,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117120265918951312541">vladimir bussi</a>'],
     'photo_reference': 'AeJbb3eSGfR8VZef3JWKtCEv_RJ6KP8ZrZYzE2Gbd4fn2zFJKoQwiwwn29O3zRN1aPx-Eaf7Pr53ZuG3auxbCUOXQjgp_uo5JdzsyPTG2cnGKviBwETvsgVp8yvQcL3IYEgEhSXxLFSD2lDcMCji2zgTP8kwzfrXhW4P5Dtgenx34UUFsAx3',
     'width': 4032}],
   'place_

In [12]:
client.detail(place_id='ChIJYdV4tBPFw0cRt33up0gVS2s')

{'html_attributions': [],
 'result': {'formatted_address': 'Av. De Fré 205, 1180 Uccle, Belgium',
  'formatted_phone_number': '0491 22 90 91',
  'name': 'La Donna',
  'rating': 4.8},
 'status': 'OK'}